<span type="title">存储层次结构</span> | <span type="update">2018-07-17</span> | <span type="version">1</span>

<span type="intro"><p type="card-text">本章介绍SRAM和DRAM。</p></span>

# 存储层次结构概况

存储器的性能显而易见的会影响到计算机的性能，如果没有Cache，那么对于单周期处理器，其必须进行硬件停顿以等待数据写入（取指、执行、回写阶段），因此造成CPU周期成倍增加。

即便只有DRAM，其速度大约为CPU的1/100，那么性能损失接近100倍。更不用提流水线处理器，在这里，中断更为复杂。

![](w7p1.png)

如上图所示，CPU在30年来性能提升了2500倍，但是DRAM则仅仅为9倍，磁盘为29倍，SRAM提升幅度相比CPU亦然太小，200倍，不过对于其余部件来言，也是非常巨大的了。因此，当代计算机最大问题就是I/O阻塞的问题，而不是CPU性能问题。

实际上，从80386开始，已经有了片外的Cache，到了80486，Cache被集成到CPU内，从8KB到现在的32KB（不能增长过快的原因是流水线对速度的要求极高，容量必然带来速度的下降）。

# DRAM

## DRAM构成

![](w7p2.png)

对于内存而言，现在指的一般是DRAM，动态随机访问存储(D-R-A-M)。其构成如上图所示。对于PC，其内存模组主要有数个DRAM芯片组成，而对于手机等便携设备，DRAM直接焊接在SOC上。对于DRAM芯片，其包含行列选择器，数据输入输出口，缓存器和内存阵列，每个行列对应数个bit的原件，每个原件由一个电容和数条连线构成，每个cell可能包含不同bit的原件。

![](w7p3.png)

如上图所示的是一个bit的原件，其接受行列选择信号，当需要写入数据时，行列信号连通，数据输入1，经过刷新放大器，对电容C进行充电。当数据读出时，同样逆过程。

需要注意，对于DRAM，因为电容回漏电，因此需要一定时间进行刷新补充电荷。

## DRAM流程

![](w7p4.png)

如上所示，是DRAM的操作流程。CPU通过总线向内存控制器发送信号，内存控制器经过BUS通过Row选择器选择行，然后Memory Array将这一行所有数据信号都发送到Sense Amplifiers缓冲区，然后内存控制器再要求Column列选择器选择信号，其在缓冲区选择后经过Data In/oUT bUFFERS进行输出。

![](w7p5.png)

在这个过程中，从行选择到列选择之间因为要写入到缓冲区，所以有一个延时，称之为tRCD（PC133标准：2~3个时钟周期，约15~23ns）。

从列地址到读取数据的延迟，称之为CL（PC133标准：2~3个时钟周期，约15~23ns）。

之后读到数据的时候，每个数据块需要耗费1个时钟周期。因为不一定需要这一行，那么就要关闭这一行，称之为行预充电。

从行预充电（关闭行）到下一次行地址输入的延迟为tRP（PC133标准：2~3个时钟周期，约15~23ns）。

## DRAM发展

DRAM在1993年被改进为SDRAM,在2000年被改进为DDR SDRAM。其中SDRAM又称之为 SDR SDRAM，DDR SDRAM 称之为 DDR1 SDRAM。

DRAM的等效时钟频率为1600Mhz，意味着其一个周期需要0.625ns。其传输带宽为1600Mhz×64bit/8=12.8GB/s（所需次数×每次数据宽度64/8：内存模组/芯片个数）

影响SDRAM的关键因素在于电容充放电时间，这个因素是难以改进的。

对于 Double Data Rate SDRAM 而言，其实现的方式如下：

首先，对于SDR SDRAM，其每一个行/列选择的cell包含8bit Unit，而对于DDR SDRAM而言，其包含16bit Unit。

对于SDR而言，其在每个时钟的上升沿传输数据，在下降沿不进行操作。而对于DDR，其在每次上升沿和下降沿都传输数据，因此数据传输量为2倍。

![](w7p6.png)

如上图所示，其数据传输速率（每次要求读写放出的数据量）提高了一倍，但是核心频率和I/O频率没有变化。

![](w7p7.png)

就其内部而言，之前每周其可以放出1个数据，现在每周期可以放出2个数据。

注意，带宽的计算方法是使用等效频率计算的：200Mhz*64/8 = 1.6 Gb/s

这里的问题在于，我们需要一些数据，但是DDR给了我们更多的数据，它生成效率提高了2倍，但却不是真实的效率提高两倍，只是顺手干了别的活。

DDR2 SDRAM 相比较 DDR SDRAM 而言，其原理如下， 将I/O频率也提升了2倍，数据传输速率提升了2倍。

![](w7p8.png)

可以看到，其包含多条线链接到Memory Array，并同时从中读取数据进行I/O，因此，I/O频率提高带动了数据传输速率的提高。相当于提升了吞吐率（而从SDR到DDR可以看作提升了效率）

以下是从SDR到DDR到DDR2的每部分耗时标准：

![](w7p9.png)

![](w7p11.png)

![](w7p10.png)

可以看到，tRCD并没有显著减少，这取决于电容放电，而数据存取的效率和吞吐率都大幅提高了。但总的来说，处理器在30年性能提升了10000倍，而DRAM的